<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Notebooks/Llama2_prod_r8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Llama2 Production

SUMA 08/10/2023

#Instructions

Running code on Google Colab
1.   Create HuggingFace account
2.   Gain access to model: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
3.   Create folder 'data' in project folder on personal Google Drive
4.   Save data set CrisiLexT26 in folder 'data'
5.   Data sourced from: https://crisislex.org/data-collections.html#CrisisLexT26
6.   Mount Google Drive
7.   Update code with Paths
8.   Set system up as per steps below
9.   Run Experiments
10.  Evaluate

## Setup

In [1]:
# Install packages
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import Packages
from langchain import PromptTemplate,  LLMChain
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
import pandas as pd
import numpy as np
import json
import os
import csv
import time
import re
import matplotlib.pyplot as plt
import json
import pycountry



In [3]:
# Mounting GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.


In [4]:
# Logging onto HuggingFace
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Preparing data

In [5]:
# Read (timestamps)

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder
def process_folder(folder_path):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'period' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        subfolder_data = process_folder(subfolder_path)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_p = pd.concat(combined_data, ignore_index=True)


# Remove spaces from column names
combined_df_p.columns = combined_df_p.columns.str.replace(' ', '')

# Rename Columns in prep for left join
combined_df_p.rename(columns={'Tweet-ID': 'Tweet ID'}, inplace=True)

# # Save the combined DataFrame to a CSV file
# output_file = 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)

print(f"Combined timestamps read")


Combined timestamps read


In [6]:
# Read tweets

# Path to the main folder
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Function to process CSV files in a folder, add folder name as label, and add subfolder name as a column
def process_folder(folder_path, label, subfolder_name):
    dfs = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv') and 'labeled' in file.lower():
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                df['Label'] = label
                df['subfolder_name'] = subfolder_name  # Add subfolder name as a column
                dfs.append(df)
    return dfs

# Read and process each subfolder
combined_data = []
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        label = subfolder
        subfolder_data = process_folder(subfolder_path, label, subfolder)
        combined_data.extend(subfolder_data)

# Concatenate all data into one DataFrame
combined_df_l = pd.concat(combined_data, ignore_index=True)

# Print the first few rows of the combined DataFrame
print(f"Combined tweets read")
combined_df_l.head(10)


Combined tweets read


,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,subfolder_name
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013_Alberta_floods
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods
5,347793818403344384,RT @weathernetwork: RT: Residents of #HighRive...,Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
6,347802211222433792,RT @BrockWHarrison: . @ElectDanielle and other...,NGOs,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods
7,347804916514951168,Lots of #abflood updates on our liveblog http:...,Media,Other Useful Information,Related and informative,2013_Alberta_floods,2013_Alberta_floods
8,347808955663257600,RT @aldjohnmar: Our first Emergency Shelter wi...,NGOs,Infrastructure and utilities,Related and informative,2013_Alberta_floods,2013_Alberta_floods
9,347812738900320256,'Move to high ground:' Rapid flooding forces e...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods


In [7]:
# Extract earliest timestamp

# Convert it to datetime first
combined_df_p['Timestamp'] = pd.to_datetime(combined_df_p['Timestamp'])

# Group by 'Tweet-ID' and aggregate to find the earliest timestamp
earliest_timestamps = combined_df_p.groupby('Tweet ID')['Timestamp'].agg('min').reset_index()

# Display or use the resulting DataFrame 'earliest_timestamps'
# print(earliest_timestamps)

In [8]:
# Left join

# Perform a left join on 'Tweet ID'
result_df = combined_df_l.merge(earliest_timestamps, on='Tweet ID', how='left')

# Display
# print(result_df)
# result_df.shape
result_df.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,subfolder_name,Timestamp
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 12:05:25+00:00
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 17:22:10+00:00
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 18:13:07+00:00
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 18:29:19+00:00
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013_Alberta_floods,2013-06-20 19:09:50+00:00


In [9]:
# Read

# Define the main folder path
main_folder = '/content/gdrive/MyDrive/iLab2/data/CrisisLexT26'

# Initialize empty lists to store the extracted data
subfolder_names = []
names = []
start_days = []
durations = []
countries = []
location_descriptions = []
sub_categories = []
types = []

# Loop through the subfolders
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)

    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        json_files = [f for f in os.listdir(subfolder_path) if f.endswith('.json')]

        # Loop through JSON files in the subfolder
        for json_file in json_files:
            json_path = os.path.join(subfolder_path, json_file)

            # Read the JSON file
            with open(json_path, 'r') as f:
                data = json.load(f)

                # Extract the required information
                subfolder_names.append(subfolder)
                names.append(data['name'])
                start_days.append(data['time']['start_day'])
                durations.append(data['time']['duration'])
                countries.append(data['location']['country'])
                location_descriptions.append(data['location']['location_description'])
                sub_categories.append(data['categorization']['sub_category'])
                types.append(data['categorization']['type'])

# Create a DataFrame from the extracted data
df_annotation = pd.DataFrame({
    'subfolder_name': subfolder_names,
    'name': names,
    'start_day': start_days,
    'duration': durations,
    'country': countries,
    'location_description': location_descriptions,
    'sub_category': sub_categories,
    'type': types
})

# Display the DataFrame
df_annotation.head()


,subfolder_name,name,start_day,duration,country,location_description,sub_category,type
0,2013_Alberta_floods,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
1,2013_Glasgow_helicopter_crash,Glasgow helicopter crash,2013-11-29,30,UK,Glasgow,Unintentional,Crash
2,2012_Costa_Rica_earthquake,Costa Rica earthquake,2012-09-04,13,Costa Rica,Costa Rica,Geophysical,Earthquake
3,2013_Manila_floods,Manila Floods,2013-08-17,11,Phillipines,Manila,Hydrological,Floods
4,2013_LA_airport_shootings,LA Airport Shootings,2013-11-01,12,US,Los Angeles,Intentional,Shootings


In [10]:
# Left join

# Perform a left join on the 'subfolder'
result_df = result_df.merge(df_annotation, left_on='subfolder_name', right_on='subfolder_name', how='left')

# Drop the duplicate 'subfolder_name'
result_df.drop(columns=['subfolder_name'], inplace=True)


In [11]:
# Count the number of rows in result_df
row_count = result_df.shape[0]

# Print the row count
print(f"Number of rows in result_df: {row_count}")

result_df.head()


Number of rows in result_df: 27933


,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013_Alberta_floods,2013-06-20 12:05:25+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 17:22:10+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 18:13:07+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013_Alberta_floods,2013-06-20 18:29:19+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013_Alberta_floods,2013-06-20 19:09:50+00:00,Alberta Floods,17/06/2013,25,Canada,Alberta,Hydrological,Floods


## Cleaning

In [12]:
# Country coding

# Funciton
def clean_country_name(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        if country:
            return country.alpha_2
        else:
            return "Unknown"
    except LookupError:
        return "Unknown"

# Clean
result_df['country'] = result_df['country'].apply(clean_country_name)


In [13]:
# Replace underscores
result_df['Label'] = result_df['Label'].str.replace('_', ' ')


In [14]:
# Show
result_df.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type
0,347686624563429378,"RT @CBCAlerts: Canmore, Alta. declares state o...",Media,Affected individuals,Related and informative,2013 Alberta floods,2013-06-20 12:05:25+00:00,Alberta Floods,17/06/2013,25,CA,Alberta,Hydrological,Floods
1,347766337344503808,RT @GlobalCalgary: If you are in #Canmore and ...,Media,Caution and advice,Related and informative,2013 Alberta floods,2013-06-20 17:22:10+00:00,Alberta Floods,17/06/2013,25,CA,Alberta,Hydrological,Floods
2,347779159327637504,RT @metrocalgary: UPDATE: Latest from the @cit...,Government,Caution and advice,Related and informative,2013 Alberta floods,2013-06-20 18:13:07+00:00,Alberta Floods,17/06/2013,25,CA,Alberta,Hydrological,Floods
3,347783236191129600,RT @GlobalCalgary: GALLERY: Incredible photos ...,Media,Not applicable,Related and informative,2013 Alberta floods,2013-06-20 18:29:19+00:00,Alberta Floods,17/06/2013,25,CA,Alberta,Hydrological,Floods
4,347793432514801665,RT @nenshi: Major risk of flooding in Calgary....,Government,Caution and advice,Related and informative,2013 Alberta floods,2013-06-20 19:09:50+00:00,Alberta Floods,17/06/2013,25,CA,Alberta,Hydrological,Floods


## Model

In [15]:
def generate_answer(text, system_prompt, instruction):
  promt_template = get_prompt(instruction, system_prompt)
  prompt = PromptTemplate(template=prompt_template, input_variables= ['text'])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  text = llm_chain.run(text)
  return text

In [16]:
# Tokenizer
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
# Load model
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [18]:
# Prompt Template Formatting
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

In [19]:
# Create a function using the code below to generate a answer for each row in the dataframe. Then apply the function to the dataframe to generate the answer and save the output to 'answer' in df.
def generate_answer(text, system_prompt, instruction):
    # # Define the instruction
    # instruction = instruction + row['TweetText']
    #define the prompt template
    prompt_template = get_prompt(instruction, system_prompt)
    #create prompt
    prompt = PromptTemplate(template=prompt_template, input_variables= ["text"])
    #create the language model chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    #generate text using the prompt template
    text = llm_chain.run(text)
    #return the text
    return text


## Experiments

In [20]:
# Grab 10 random samples
# filtered_df = result_df.query("Label == '2013_Colorado_floods'")
testing_df = result_df.sample(n=20, random_state=42)
testing_df.columns = testing_df.columns.str.replace(' ', '')
testing_df['TweetText'] = testing_df['TweetText'].astype(str)


# testing_df.head(12)


In [24]:
# Experiment 1: Crisis location code (country)

system_prompt = "You are an advance assistant that excels at analysing text that contains useful information during or after a crisis. A crisis can include bushfires, wildfires, floods, hurricanes, earthquakes, covid-19, pandemic etc. Stop after giving one answer. If the text is ambigous return  '''Not applicable'''. Only return one code that is the most appropriate without explaination. Return one label without comment or dialogugue. If the location is a state, return the country in which the state is. "
instruction = "For the following text for me the ISO country code. {text}"

# Create an empty 'answer' column in testing_df
testing_df['country_pred'] = ''

# Loop over the data
for index, row in testing_df.iterrows():
    TweetText = row['TweetText']

    # Update the 'answer' column in the DataFrame using the output from generate_answer
    testing_df.loc[index, 'country_pred'] = generate_answer(TweetText, system_prompt, instruction)



/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [21]:
# Experiment 2: crisis informativeness type labels

system_prompt = "You are a crisis researcher that excels at classifying whether the text contains useful information in relation to a crisis. Stop after giving one answer. If it is not related to a crisis return '''Not applicable'''. Only return one label that is the most appropriate. "
instruction = '''"Classify {TweetText} for me whether it is informatie and whether it is related to {Label}, using these crisis information labels:
"Related - but not informative" = TweetText contains meaningfull information but hot about Label and there is no context in to betweenn the two texts.
"Related and informative" = TweetText contains meaningfull information about Label and there is context in to betweenn the two texts.
"Not related" =  TweetText has nothing to do with Label and there is no context in to betweenn the two texts.


Return one label with explaination."'''


# Create an empty 'answer' column in testing_df
testing_df['Informativeness_pred'] = ''

# Loop over the data
for index, row in testing_df.iterrows():

    # Input variables
    input_variables = {
        "TweetText": row['TweetText'],
        "Label": row['Label']
        }

    #define the prompt template
    prompt_template = get_prompt(instruction, system_prompt)

    #create prompt
    prompt = PromptTemplate(template=prompt_template, input_variables= ["TweetText", "Label"])

    #create the language model chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    #generate text using the prompt template
    text_inf = llm_chain.run(input_variables)

    # Update the 'answer' column in the DataFrame using the output from generate_answer
    testing_df.loc[index, 'Informativeness_pred'] = text_inf


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
# crisis information type labels v9 [with explaination in system prompt][no label list][label in quotation marks][text in quotation marks]

system_prompt = "You are a crisis researcher that excels at classifying whether the text contains useful information in relation to a crisis. Stop after giving one answer. If it is not related to a crisis return '''Not applicable'''. Only return one label that is the most appropriate. "
instruction = '''"Classify using these following labels which are defined as follows:
"Caution and advice for residents"= Warnings issues or lifted, guidance, and tips.
"Affected residents"= Casualties due to the crisis. Questions and/or reports about missing, trapped, displaced, evacuated or found people.
"Infrastructure and utilities damage"= Buildings or roads damaged or operational; utilities/services interrupted or restored.
"Donations or volunteering"= Needs or donations of supplies such as food, water, clothing, medical supplies or blood. Services needed or offered by volunteers or professionals.
"expressing Sympathy and support for affected"= Status updates about individuals or loved ones; emotional support, thoughts and prayers.
"other useful information"= Other information that is useful to a crisis.
"Not applicable"= No information related to a crisis.

Return one label with explaination.
"{text}" "'''

# Create an empty 'answer' column in testing_df
testing_df['Pred_InformationType_v9'] = ''

# Loop over the data
for index, row in testing_df.iterrows():
    TweetText = row['TweetText']

    # Generate the prompt
    # prompt_template = get_prompt(instruction, system_prompt)

    # Update the 'answer' column in the DataFrame using the output from generate_answer
    testing_df.loc[index, 'Pred_InformationType_v9'] = generate_answer(TweetText, system_prompt, instruction)

testing_df.head(20)

In [25]:
# Check
testing_df.head(20)


,TweetID,TweetText,InformationSource,InformationType,Informativeness,Label,Timestamp,name,start_day,duration,country,location_description,sub_category,type,Informativeness_pred,Pred_InformationType_v9,country_pred
17799,239986309865807872,RT @garcilasop: #Lafuncióndebecontinuar es una...,Outsiders,Other Useful Information,Related - but not informative,2012 Venezuela refinery,2012-08-27 07:22:49+00:00,Venezuela refinery explosion,2012-08-24,12,Unknown,Falcon,Unintentional,Explosion,"Based on the text provided, I would classify...","Label: ""Other useful information""\nExplanati...",VEN
9419,275898683109801984,Typhoon Bopha Pounds Southern Philippines: A p...,Media,Caution and advice,Related and informative,2012 Typhoon Pablo,2012-12-04 09:45:46+00:00,Typhoon Pablo,2012-11-24,21,Unknown,Phillipines,Meteorological,Typhoon,"Based on the provided tweet, I would classif...","Label: ""Affected residents""\nExplanation: Th...",PHP
1972,412933108561108992,#helicopter #parachute #drop #sarmaaaan http:/...,Not labeled,Not labeled,Not related,2013 Glasgow helicopter crash,2013-12-17 13:11:39+00:00,Glasgow helicopter crash,2013-11-29,30,Unknown,Glasgow,Unintentional,Crash,"Based on the tweet you provided, I would cla...",Not applicable. The tweet contains no inform...,Not applicable
24446,392992116478996481,RT @MarkDiStef: Defence Dept's been investigat...,Outsiders,Caution and advice,Related and informative,2013 Australia bushfire,2013-10-23 12:33:16+00:00,Australia wildfires,2013-10-12,21,AU,New South Wales,Climatological,Wildfire,"Based on the given tweet, I would classify i...","Label: ""Other useful information""\nExplanati...",Not applicable
12125,327400365765046272,Tak Ada WNI Korban Runtuhnya Rana Plaza Bangla...,Media,Not applicable,Related and informative,2013 Savar building collapse,2013-04-25 12:35:04+00:00,Savar building collapse,2013-04-23,36,BD,Savar,Unintentional,Collapse,"Based on the provided tweet text, I would cl...","Label: ""Affected residents""\nExplanation: Th...",BD
27873,404735664329351168,RT @nicola_pinna: A #Olbia c'è bisogno di cib...,Outsiders,Donations and volunteering,Related and informative,2013 Sardinia floods,2013-11-24 22:17:56+00:00,Sardinia Floods,2013-11-16,13,IT,Sardinia,Hydrological,Floods,"Based on the text you provided, I would clas...","Label: ""Donations or volunteering""\nExplanat...",Sardinia (IT)
2055,415154554549075968,RT @BreahnaZhane: RIP To Our Fort Riley Soldie...,Not labeled,Not labeled,Not related,2013 Glasgow helicopter crash,2013-12-23 16:18:53+00:00,Glasgow helicopter crash,2013-11-29,30,Unknown,Glasgow,Unintentional,Crash,"Based on the provided tweet, I would classif...",Label: Not applicable\nThis tweet does not c...,US
15053,390643910214946817,#Earthquake 7.2 tuesday 15 oct destroyed a lot...,Media,Infrastructure and utilities,Related and informative,2013 Bohol earthquake,2013-10-17 01:02:20+00:00,Bohol earthquake,2013-10-14,12,Unknown,Phillipines,Geophysical,Earthquake,"Based on the provided text, I would classify...","Label: ""Affected residents""\nExplanation: Th...","Based on the text provided, the most appropr..."
4667,396330455303475200,RT @jackmooring: Praying for the victims at LA...,Outsiders,Sympathy and support,Related - but not informative,2013 LA airport shootings,2013-11-01 17:38:38+00:00,LA Airport Shootings,2013-11-01,12,Unknown,Los Angeles,Intentional,Shootings,"Based on the given text, I would classify it...",Label: Not applicable\nThis tweet does not c...,Not applicable
8365,302496073392459777,"RT @TeamCoco: OMG guyz, did you see that meteo...",Media,Not applicable,Related - but not informative,2013 Russia meteor,2013-02-15 19:14:18+00:00,Russian meteor,2013-02-14,19,Unknown,Chelyabinsk,Others,Meteorite,"Based on the text you provided, I would clas...",Not applicable. The tweet does not contain a...,RU


In [ ]:
answer_1 = testing_df.iloc[1, 15]
print(answer_1)


answer_1 = testing_df.iloc[1, 4]
print(answer_1)


  Based on the information provided, I would classify the text as "Related and informative". The text mentions a powerful typhoon pounding the southern Philippines, which is a clear indication of a crisis situation. The mention of Typhoon Pablo in the tweet also suggests that the text is related to a previous crisis event. Therefore, I would label the text as "Related and informative".
Related and informative


In [ ]:
#Parse text function
InfoTypeLabels =  [ "Caution and advice for residents", "Written by Affected individuals", "Infrastructure and utilities damage", "Soliciting Donations or volunteering to help", "expressing Sympathy and support for affected", "other useful information", "Not applicable"]

def parse_label(text):
    colon_index = text.find("Label:")
    if colon_index != -1:
        second_colon_index = text.find(":", colon_index + 6)
        if second_colon_index != -1:
            substring = text[colon_index + 6:second_colon_index]
            for label in InfoTypeLabels:
                if label.lower() in substring.lower():
                    return label
    for label in InfoTypeLabels:
        if label.lower() in text.lower():
            return label
    value = re.findall(r'"(.*?)"', text)
    if value:
        return value[0]
    return "Error"


testing_df['Parsed_Pred_InformationType_v9'] = testing_df['Pred_InformationType_v9'].apply(parse_label)
testing_df['Parsed_Pred_InformationType_Baseline'] = testing_df['Pred_InformationType_Baseline'].apply(parse_label)
# Dict to rename the label to the original label name
dict ={"Not applicable" : 'Not labeled',"other useful information" :'Other Useful Information',"Caution and advice for residents" :'Caution and advice',"Affected residents" :'Affected individuals',"Infrastructure and utilities damage" :'Infrastructure and utilities',"Soliciting Donations or volunteering to help" :'Donations and volunteering',"Donations or volunteering" :'Donations and volunteering',"expressing Sympathy and support for affected" :'Sympathy and support', "Warnings and advice for residents." :'Caution and advice'}
testing_df2= testing_df.replace({"Parsed_Pred_InformationType_v9": dict, "Parsed_Pred_InformationType_Baseline": dict})


testing_df2.head(100)

## Performance Evaluation